# Circuit extraction heterogeneous networks


For heteroegeneous networks with general topologies, we need to ensure the output circuit only requests EPR pairs on directly connected nodes. This means all entanglement swapping or long-range entanglement generation is handled within QPUs using auxiliary links. This simplifies the networking level, as only suaxiliary EPR pairs are requested.

The most entanglement-efficient techniques exploit multi-gate teleportation based on cat-etnanglements which temporarily store non-local copies of qubits. By default our partitioning methods exploit this. However, this can lead to large demands for communication qubits to store linked copies. The circuit extractor dynamically allocates communication qubits for this, and so can estimate the requirements. However, if the user wishes to restrict this, they may avoid using cat-entanglements by setting ```group_gates=False``` when building the ```QuantumCircuitHyperGraph``` object. This will ensure minimal use of communication qubits, employing only basic gate teleportation and state teleportation.

In [ ]:
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits import cp_fraction
from qiskit import transpile
from disqco import QuantumNetwork, QuantumCircuitHyperGraph, set_initial_partition_assignment
from disqco.parti import FiducciaMattheyses
# %load_ext jupyter_tikz if displaying tikz figures 
num_qubits = 32

circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5, seed=8360)

num_partitions = 4  # Define the number of partitions or QPUs you will distribute over
depth = circuit.depth()

qpu_size = num_qubits // num_partitions + 1  # Define the number of qubits per QPU. For simplicity, we divide qubits evenly and add one additional space to each for teleportation (an extra space is needed for the teleportation qubit so the algorithmr requires it!)
qpu_sizes = [qpu_size] * num_partitions # Store the number of qubits per QPU in a list

# Create a quantum network with the specified number of qubits and partitions

# If we do not specificy the connectivity, we have all-to-all by default.

quantum_network = QuantumNetwork.create(qpu_sizes=qpu_sizes,coupling_type='linear')
graph = QuantumCircuitHyperGraph(circuit, group_gates=True)

print(f'Number of qubits in circuit {circuit.num_qubits}')
print(f'Circuit depth: {circuit.depth()}')

FM_partitioner = FiducciaMattheyses(circuit=circuit,
                                    hypergraph=graph,
                                    network=quantum_network,
                                    graph=graph,
                                    initial_assignment=set_initial_partition_assignment(graph, quantum_network),
                                    )
quantum_network.draw()

In [ ]:
from disqco.coarsening import HypergraphCoarsener

coarsener = HypergraphCoarsener()

coarseneing_method = coarsener.coarsen_recursive_batches_mapped

results = FM_partitioner.multilevel_partition(coarsener=coarseneing_method)

print(f'Best cost: {results['best_cost']}')


In [ ]:
graph.draw(network=quantum_network,
           assignment=results['best_assignment'],
           output="mpl",
           show_labels=False,
           dpi=150)

In [ ]:
from disqco import PartitionedCircuitExtractor

extractor = PartitionedCircuitExtractor(graph=graph, 
                                        network=quantum_network, 
                                        partition_assignment=results['best_assignment'])

extracted_circuit = extractor.extract_partitioned_circuit()

In [ ]:
print(f' Number of EPR pairs in circuit: {extracted_circuit.count_ops().get("EPR", 0)}')

extracted_circuit.remove_final_measurements()
if extracted_circuit.num_qubits <= 20:
    fig = extracted_circuit.draw(style='bw',output='mpl', cregbundle=True, fold=100)
    display(fig)

In [ ]:
from disqco.circuit_extraction.verification import check_no_cross_partition_gates

is_valid = check_no_cross_partition_gates(extracted_circuit, quantum_network.qpu_graph)
print("Circuit valid:", is_valid)